# Harmonizome ETL: CORUM

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: http://mips.helmholtz-muenchen.de/corum/#download

In [1]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

/home/charlesdai/Projects/Harmonizome-Data-Processing-Appyters


In [2]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [4]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [5]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

This notebook was run on: 2020-07-06 
Python version: 3.8.0 (default, Oct 28 2019, 16:14:01) 
[GCC 8.3.0]


# Initialization

In [6]:
%%appyter hide_code

{% do SectionField(
    name='data',
    title='Upload Data',
    img='load_icon.png'
) %}

In [10]:
%%appyter code_eval

{% do DescriptionField(
    name='description',
    text='The example below was sourced from <a href="http://mips.helmholtz-muenchen.de/corum/#download" target="_blank">mips.helmholtz-muenchen.de/corum</a>. If clicking on the example does not work, it should be downloaded directly from the source website.',
    section='data'
) %}

{% set df_file = FileField(
    constraint='.*\.txt.zip$',
    name='all_complexes', 
    label='Complexes (txt.zip)', 
    default='Input/CORUM/allComplexes.txt.zip',
    examples={
        'allComplexes.txt.zip': 'http://mips.helmholtz-muenchen.de/corum/download/allComplexes.txt.zip'
    },
    section='data'
) %}


### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

### Output Path

In [8]:
output_name = 'corum'

path = 'Output/CORUM'
if not os.path.exists(path):
    os.makedirs(path)

# Load Data

In [12]:
%%appyter code_exec

df = pd.read_csv(
    {{df_file}}, 
    sep='\t', usecols=['ComplexName', 'Organism', 'subunits(Gene name)'])

```python
df = pd.read_csv(
    'Input/CORUM/allComplexes.txt.zip',
    sep='\t', usecols=['ComplexName', 'Organism', 'subunits(Gene name)'])
```

In [13]:
df.head()

,ComplexName,Organism,subunits(Gene name)
0,BCL6-HDAC4 complex,Human,BCL6;HDAC4
1,BCL6-HDAC5 complex,Human,BCL6;HDAC5
2,BCL6-HDAC7 complex,Human,BCL6;HDAC7
3,Multisubunit ACTR coactivator complex,Human,EP300;CREBBP;KAT2B;NCOA3
4,6S-nuclear aryl hydrocarbon (Ah) receptor liga...,Mouse,Ahr;Arnt


In [ ]:
df.shape

# Pre-process Data

## Get Relevant Data

In [ ]:
# Get only relevant species
df = df[np.logical_or.reduce([
    df['Organism'] == 'Human',
    df['Organism'] == 'Mouse',
    df['Organism'] == 'Rat',
])]
df.head()

## Split Subunit Genes

In [ ]:
df['Gene Symbol'] = df['subunits(Gene name)'].map(lambda s: s.split(';'))
df = df[['Gene Symbol', 'ComplexName']]
df = df.explode('Gene Symbol').set_index('Gene Symbol')
df.columns = ['Complex Name']
df.head()

# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
df = uf.map_symbols(df, symbol_lookup, remove_duplicates=True)
df.shape

# Analyze Data

## Create Binary Matrix

In [ ]:
binary_matrix = uf.binary_matrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.save_data(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

## Create Gene List

In [ ]:
gene_list = uf.gene_list(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(binary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(binary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(binary_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)